# Задание:    

1. Посчитать OnTime - Метрика своевременности доставки. (Доставили в плановую дату доставки или нет), в %

 - Необходимо видеть изменение метрики по дням;

 - Необходимы разрезы по перевозчикам, по типу доставки, по кластерам склада, по таймслотам;

2. Посчитать PromisedClick2Delivery - метрика обещанного срока доставки (от даты заказа до первой плановой даты доставки), в сутках.

 - Необходима динамика по дням, неделям, месяцам;

 - Необходимо сравнение перевозчиков;

 - Необходимо понимание уровня метрики из кластера в кластер.

3. Посчитать Click2Delivery - метрика фактического срока доставки (от даты заказа до момента вручения), в сутках.

 - Необходимо сравнение с метрикой PromisedClick2Delivery;

 - Необходимо понимание кластеров, где заказы доставляются быстрее, чем мы обещаем клиентам;

4. Посчитать GMV Accepted - метрика совокупного объема продаж (считается по заказам, оформленным в указанный день), в рублях.

 - Необходимо видеть изменение метрики по дням;

 - Необходимы разрезы по перевозчикам, по типу доставки, по кластерам склада;

5. Посчитать Return Rate - доля возвратов (отношение количества возвратов к заказам в этот день), в %.

 - Необходимо видеть изменение метрики по дням;

 - Необходимы разрезы по перевозчикам, по кластерам склада и клиента;

6. Посчитать GMV D-R - метрика, отражающая совокупный объем доставленных заказов за вычетом возвратов, в рублях.

 - Необходимо видеть изменение метрики по дням (для всех дней, которые были раньше, чем дата расчета метрики);

 - Необходимы разрезы по перевозчикам, по типу доставки, по кластерам склада.

Описание таблиц

Все данные находятся в табличном виде в Clickhouse

Таблица logistic_online содержит данные о логистике.

date_order - дата, в которую клиент оформляет заказ

number_order - уникальный номер заказа клиента (в столбце номер может повторяться, т.к. заказы переносятся и имеют раные даты доставки)

timeslot_number - нумерация кол-ва попыток доставить в плановую дату доставки. Если доставка не была осуществлена, то заказ переносится на новую дату

result_data_order - дата, в которую заказ был доставлен клиенту

delivery_type - выбранный клиентом способ получения заказа (курьерка или самовывоз)

carrier_name - перевозчик, которым заказ доставляется клиенту

seller_ID - уникальный идентификатор продавца

order_status - последний актуальный статус заказа (фиксируется во всех строках последний статус заказа)

customer_cluster - региональная принадлежность клиента

warehouse_cluster - региональная принадлежность продавца

Таблица product_online содержит данные о товарах в заказе.

number_order - уникальный номер заказа клиента (в столбце номер может повторяться, т.к. так как в одном заказе может быть несколько товаров)

product_ID - уникальный номер товара (В столбце номер может повторяться, т.к. так как один товар может быть в заказах нескольких клиентов)

product_price - стоимость одной единицы товара в рублях

amount - количество единиц товара в заказе

Таблица returns_online содержит данные о возвратах

number_order - уникальный номер заказа клиента (в столбце номер может повторяться, т.к. так как в одном заказе может быть несколько товаров)

product_ID - уникальный номер товара (в столбце номер может повторяться, т.к. так как один товар может быть в заказах нескольких клиентов)

amount_return - количество единиц товара, которые были возвращены (не больше, чем количество единиц товара, приобретенных в указаном заказе)

return_date - дата совершения возврата

Считаю все данные для выполнения задания

In [103]:
import pandas as pd
import numpy as np
import pandahouse as ph
import seaborn as sns
import matplotlib.pyplot as plt

In [104]:
connection_default = {'host': 'http://clickhouse.s',
                      'database': 'default',
                      'user': 'student',
                      'password': '2022'
                     }

In [105]:
logistics_online = '''
    SELECT
        assumeNotNull(replaceAll(date_order, '012', '12')) as date_order,
        number_order,
        assumeNotNull(replaceAll(planned_data_order, '012', '12')) as planned_data_order,
        timeslot_number, 
        assumeNotNull(replaceAll(result_data_order , '012', '12')) as result_data_order, 
        delivery_type,
        carrier_name,
        seller_ID,
        order_status,
        customer_cluster,
        warehouse_cluster
    FROM {db}.logistics_online
    
    '''

logistics_online = ph.read_clickhouse(query = logistics_online, connection = connection_default)
logistics_online.head()

,date_order,number_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster
0,2021-5-01,313655114,2021-14-01,1,2021-11-01,Самовывоз,3PL 1,301202,Доставлен,Кластер Краснодар,Кластер Москва
1,2021-3-01,315229153,2021-11-01,1,2021-8-01,Курьер,Самодоставка,40560,Доставлен,Кластер Московская область,Кластер Московская область
2,2021-8-01,3114106095,2021-16-01,1,2021-14-01,Курьер,Самодоставка,40560,Доставлен,Кластер Московская область,Кластер Московская область
3,2021-12-01,31145541407,2021-20-01,1,2021-15-01,Самовывоз,3PL 1,301202,Доставлен,Кластер Санкт-Петербург,Кластер Москва
4,2021-16-01,325050136,2021-26-01,1,2021-25-01,Самовывоз,3PL 1,946233,Доставлен,Кластер Санкт-Петербург,Кластер Санкт-Петербург


In [106]:
product_online = '''
    SELECT 
        number_order,
        product_ID,
        product_price,
        amount 
    FROM default.product_online 
'''

product_online = ph.read_clickhouse(query = product_online, connection = connection_default)
product_online.head()

,number_order,product_ID,product_price,amount
0,313655114,983587801,"5,512.44",1
1,313655114,983587158,"4,070.35",2
2,313655114,983582669,"3,591.97",1
3,315229153,983589424,"4,433.23",3
4,315229153,983587827,"2,862.50",2


In [107]:
returns_online = '''
SELECT 
    number_order,
    product_ID,
    amount_return,
    return_date 
from  default.returns_online
'''

returns_online = ph.read_clickhouse(query = returns_online, connection = connection_default)
returns_online.head(10)

,number_order,product_ID,amount_return,return_date
0,3031410942,983588011,1,1970-01-01
1,31014374140,983586554,1,2021-07-01
2,3147145173,983589156,1,2021-11-01
3,3141464669,983589362,1,2021-11-01
4,3131450699,983589729,3,2021-12-01
5,3131450699,983589729,3,2021-12-01
6,315774746,983585035,1,2021-12-01
7,315195519,983587637,1,1970-01-01
8,3155331412,983580141,1,1970-01-01
9,3155331412,983580141,1,1970-01-01


In [108]:
logistics_online.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6597 entries, 0 to 6596
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   date_order          6597 non-null   object
 1   number_order        6597 non-null   int64 
 2   planned_data_order  6597 non-null   object
 3   timeslot_number     6597 non-null   int8  
 4   result_data_order   6597 non-null   object
 5   delivery_type       6597 non-null   object
 6   carrier_name        6597 non-null   object
 7   seller_ID           6597 non-null   int64 
 8   order_status        6597 non-null   object
 9   customer_cluster    6597 non-null   object
 10  warehouse_cluster   6597 non-null   object
dtypes: int64(2), int8(1), object(8)
memory usage: 522.0+ KB


In [109]:
logistics_online.nunique()

date_order             115
number_order          5976
planned_data_order     106
timeslot_number          7
result_data_order       81
delivery_type            2
carrier_name             5
seller_ID              167
order_status             5
customer_cluster        12
warehouse_cluster       11
dtype: int64

Количество number_order 6597, количество уникальных номеров заказов - 5976. Проверю причину, рассмотрев детально некоторые заказы


In [110]:
logistics_online.number_order.value_counts()

35214401456     7
33114014319     5
355157030       5
3553149262      4
355359296       4
               ..
3465129143      1
3400361146      1
339145143141    1
344749244       1
361144320       1
Name: number_order, Length: 5976, dtype: int64

Рассмотрю более детально несколько заказов, где число записей более 1

In [111]:
logistics_online.query("number_order=='35214401456'")

,date_order,number_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster
1029,,35214401456,2021-11-03,4,,Курьер,3PL 1,43292,На пути к клиенту,Кластер Краснодар,Кластер Санкт-Петербург
1030,2021-27-02,35214401456,2021-5-03,1,,Курьер,3PL 1,43292,На пути к клиенту,Кластер Краснодар,Кластер Санкт-Петербург
1031,,35214401456,2021-13-03,5,,Курьер,3PL 1,43292,На пути к клиенту,Кластер Краснодар,Кластер Санкт-Петербург
1032,,35214401456,2021-16-03,7,,Курьер,3PL 1,43292,На пути к клиенту,Кластер Краснодар,Кластер Санкт-Петербург
1033,,35214401456,2021-9-03,2,,Курьер,3PL 1,43292,На пути к клиенту,Кластер Краснодар,Кластер Санкт-Петербург
1034,,35214401456,2021-15-03,6,,Курьер,3PL 1,43292,На пути к клиенту,Кластер Краснодар,Кластер Санкт-Петербург
1035,,35214401456,2021-10-03,3,,Курьер,3PL 1,43292,На пути к клиенту,Кластер Краснодар,Кластер Санкт-Петербург


In [112]:
logistics_online.query("number_order=='3553149262'")

,date_order,number_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster
5467,,3553149262,2021-16-03,4,2021-15-03,Курьер,Самодоставка,33900,Доставлен,Кластер Краснодар,Кластер Новосибирск
5468,,3553149262,2021-10-03,2,2021-15-03,Курьер,Самодоставка,33900,Доставлен,Кластер Краснодар,Кластер Новосибирск
5469,,3553149262,2021-13-03,3,2021-15-03,Курьер,Самодоставка,33900,Доставлен,Кластер Краснодар,Кластер Новосибирск
5470,2021-28-02,3553149262,2021-5-03,1,2021-15-03,Курьер,Самодоставка,33900,Доставлен,Кластер Краснодар,Кластер Новосибирск


Становится понятно, что в случае более чем 1 попытки доставить заказ, дата заказа не отображается. В случае успешной доставки, дата отображается одинаковая во всех строках по данному заказу

1. Посчитать OnTime - Метрика своевременности доставки. (Доставили в плановую дату доставки или нет), в %

 - Необходимо видеть изменение метрики по дням;

 - Необходимы разрезы по перевозчикам, по типу доставки, по кластерам склада, по таймслотам;

In [113]:
logistics_online['date_order'].unique()

array(['2021-5-01', '2021-3-01', '2021-8-01', '2021-12-01', '2021-16-01',
       '2021-2-02', '2021-28-01', '2021-4-02', '2021-5-02', '',
       '2021-9-02', '2021-11-02', '2021-7-02', '2021-14-02', '2021-8-02',
       '2021-20-02', '2021-21-02', '2021-8-03', '2021-17-02',
       '2021-23-02', '2021-4-03', '2021-2-03', '2021-9-03', '2021-11-03',
       '2021-28-02', '2021-13-03', '2021-3-03', '2021-12-03',
       '2021-17-03', '2021-2-01', '2021-6-01', '2021-10-01', '2021-15-01',
       '2021-23-01', '2021-24-01', '2021-30-01', '2021-31-01',
       '2021-6-02', '2021-3-02', '2021-15-03', '2021-12-02', '2021-10-02',
       '2021-19-02', '2021-24-02', '2021-26-02', '2021-25-02',
       '2021-5-04', '2021-1-03', '2021-14-03', '2021-10-03', '2021-7-03',
       '2021-16-03', '2021-4-01', '2021-19-01', '2021-20-01',
       '2021-25-01', '2021-27-01', '2021-1-02', '2021-13-02',
       '2021-15-02', '2021-22-02', '2021-5-03', '2021-27-02',
       '2021-11-01', '2021-17-01', '2021-29-01', '2021

В наименовании дат есть ошибки в записи, нужно исправить

In [114]:
logistics_online['date_order']=logistics_online['date_order'].replace('2021-01-002','2021-01-02')

In [115]:
# все столбцы с датами - в формате ' object', приведу в формат даты

In [116]:
logistics_online['date_order'] = pd.to_datetime(logistics_online['date_order'], format="%Y-%d-%m", errors='coerce')
logistics_online['planned_data_order'] = pd.to_datetime(logistics_online['planned_data_order'], format="%Y-%d-%m", errors='coerce')
logistics_online['result_data_order'] = pd.to_datetime(logistics_online['result_data_order'], format="%Y-%d-%m", errors='coerce')

In [117]:
logistics_online.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6597 entries, 0 to 6596
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   date_order          5972 non-null   datetime64[ns]
 1   number_order        6597 non-null   int64         
 2   planned_data_order  6597 non-null   datetime64[ns]
 3   timeslot_number     6597 non-null   int8          
 4   result_data_order   5707 non-null   datetime64[ns]
 5   delivery_type       6597 non-null   object        
 6   carrier_name        6597 non-null   object        
 7   seller_ID           6597 non-null   int64         
 8   order_status        6597 non-null   object        
 9   customer_cluster    6597 non-null   object        
 10  warehouse_cluster   6597 non-null   object        
dtypes: datetime64[ns](3), int64(2), int8(1), object(5)
memory usage: 522.0+ KB


In [118]:
logistics_online.head()

,date_order,number_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster
0,2021-01-05,313655114,2021-01-14,1,2021-01-11,Самовывоз,3PL 1,301202,Доставлен,Кластер Краснодар,Кластер Москва
1,2021-01-03,315229153,2021-01-11,1,2021-01-08,Курьер,Самодоставка,40560,Доставлен,Кластер Московская область,Кластер Московская область
2,2021-01-08,3114106095,2021-01-16,1,2021-01-14,Курьер,Самодоставка,40560,Доставлен,Кластер Московская область,Кластер Московская область
3,2021-01-12,31145541407,2021-01-20,1,2021-01-15,Самовывоз,3PL 1,301202,Доставлен,Кластер Санкт-Петербург,Кластер Москва
4,2021-01-16,325050136,2021-01-26,1,2021-01-25,Самовывоз,3PL 1,946233,Доставлен,Кластер Санкт-Петербург,Кластер Санкт-Петербург


Переведу пропущенные значения в столбцах с датами в произвольную дату '1950-01-01', что бы не работать с NAT (если этого не сделать, в дальнейшем столкнусь с проблемой когда при фильтрации пустые ячейки теряются)

In [119]:
logistics_online=logistics_online.replace(np.nan, '1950-01-01')
logistics_online['date_order']=pd.to_datetime(logistics_online.date_order)
logistics_online['result_data_order']=pd.to_datetime(logistics_online.result_data_order)
logistics_online['planned_data_order']=pd.to_datetime(logistics_online.planned_data_order)

logistics_online.sort_values('planned_data_order').head()

,date_order,number_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster
3649,2020-12-25,303724460,2021-01-01,1,2020-12-29,Самовывоз,3PL 2,68503,Доставлен,Кластер Краснодар,Кластер Москва
1509,2020-12-24,307256994,2021-01-01,1,2021-01-04,Самовывоз,3PL 2,68503,Доставлен,Кластер Екатеринбург,Кластер Москва
6358,2020-12-23,30150141149,2021-01-01,1,2020-12-29,Самовывоз,3PL 1,2619,Доставлен,Кластер Екатеринбург,Кластер Москва
1918,2020-12-21,303621239,2021-01-01,1,2020-12-25,Самовывоз,3PL 2,68503,Доставлен,Кластер Казань,Кластер Москва
2718,2020-12-24,30665014144,2021-01-01,1,2021-01-04,Самовывоз,3PL 2,989330,Доставлен,Кластер Краснодар,Кластер Екатеринбург


Для получения данных, на основании которых буду считать метрику, проведу предварительное исследование данных

- Проверю на дубликаты

In [120]:
logistics_online.loc[logistics_online.duplicated()]

,date_order,number_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster
728,2021-02-16,344629456,2021-02-26,1,2021-03-03,Курьер,3PL 1,9504,Доставлен,Кластер Калининград,Кластер Казань


In [121]:
logistics_online.loc[logistics_online.duplicated(['number_order', 'timeslot_number'])]

,date_order,number_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster
13,2021-02-11,336693909,2021-02-16,1,2021-02-17,Самовывоз,3PL 3,2345336,Доставлен,Кластер Екатеринбург,Кластер Московская область
450,2021-03-05,357469624,2021-03-12,1,1950-01-01,Курьер,3PL 3,8612129,Отменен,Кластер Краснодар,Кластер Москва
728,2021-02-16,344629456,2021-02-26,1,2021-03-03,Курьер,3PL 1,9504,Доставлен,Кластер Калининград,Кластер Казань
6427,2021-02-06,333776216,2021-02-11,1,2021-02-13,Самовывоз,3PL 1,633129,Доставлен,Кластер Санкт-Петербург,Кластер Екатеринбург


In [122]:
logistics_online.loc[logistics_online.duplicated(['number_order', 'planned_data_order'])]

,date_order,number_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster
13,2021-02-11,336693909,2021-02-16,1,2021-02-17,Самовывоз,3PL 3,2345336,Доставлен,Кластер Екатеринбург,Кластер Московская область
450,2021-03-05,357469624,2021-03-12,1,1950-01-01,Курьер,3PL 3,8612129,Отменен,Кластер Краснодар,Кластер Москва
728,2021-02-16,344629456,2021-02-26,1,2021-03-03,Курьер,3PL 1,9504,Доставлен,Кластер Калининград,Кластер Казань
6427,2021-02-06,333776216,2021-02-11,1,2021-02-13,Самовывоз,3PL 1,633129,Доставлен,Кластер Санкт-Петербург,Кластер Екатеринбург


In [123]:
logistics_online=logistics_online.drop_duplicates()
logistics_online=logistics_online.drop_duplicates(['number_order', 'timeslot_number'])
logistics_online=logistics_online.drop_duplicates(['number_order', 'planned_data_order'])

- Проверю на правильность записи в столбце с timeslot_number

In [124]:
df_max=logistics_online.sort_values(['number_order', 'timeslot_number'], ascending=False)\
                        .drop_duplicates(subset='number_order', keep='first')

In [125]:
df_max['max_timeslot_number']=df_max['timeslot_number']

In [126]:
df_max=df_max[['number_order','max_timeslot_number']]

In [127]:
df_max=df_max.merge(logistics_online, on='number_order', how='right')
df_max.head(5)

,number_order,max_timeslot_number,date_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster
0,313655114,1,2021-01-05,2021-01-14,1,2021-01-11,Самовывоз,3PL 1,301202,Доставлен,Кластер Краснодар,Кластер Москва
1,315229153,1,2021-01-03,2021-01-11,1,2021-01-08,Курьер,Самодоставка,40560,Доставлен,Кластер Московская область,Кластер Московская область
2,3114106095,1,2021-01-08,2021-01-16,1,2021-01-14,Курьер,Самодоставка,40560,Доставлен,Кластер Московская область,Кластер Московская область
3,31145541407,1,2021-01-12,2021-01-20,1,2021-01-15,Самовывоз,3PL 1,301202,Доставлен,Кластер Санкт-Петербург,Кластер Москва
4,325050136,1,2021-01-16,2021-01-26,1,2021-01-25,Самовывоз,3PL 1,946233,Доставлен,Кластер Санкт-Петербург,Кластер Санкт-Петербург


In [128]:
df_max=df_max.groupby(['number_order', 'max_timeslot_number'], as_index=False)\
            .agg({'date_order':'count'}).sort_values('max_timeslot_number',ascending=False)\
            .rename(columns={'date_order': 'timeslot_count'})
df_max['diff']=df_max['max_timeslot_number']-df_max['timeslot_count']
df_max=df_max.query('diff!=0').sort_values('diff', ascending=False)
df_max

,number_order,max_timeslot_number,timeslot_count,diff
5077,21414577700,2,1,1
0,300232595,2,1,1
5297,33196141469,2,1,1
3682,3399142241,2,1,1
2805,3039114625,2,1,1
9,304073092,2,1,1
4,302459167,2,1,1
3,302073096,2,1,1


Вижу, что есть записи где max timeslot > количество timeslot

Рассмотрю эти заказы, что бы понимать что произошло, почему есть заказы, где идентификатор количества доставок не соответствует записям о количестве доставок

In [129]:
logistics_online.query("number_order=='21414577700' or number_order=='300232595' \
                        or number_order=='33196141469'or number_order=='3399142241' \
                        or number_order=='3039114625' or number_order=='304073092'\
                        or number_order=='302459167' or number_order=='302073096'")

,date_order,number_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster
2060,1950-01-01,33196141469,2021-02-23,2,2021-02-24,Самовывоз,3PL 2,946233,Доставлен,Кластер Краснодар,Кластер Санкт-Петербург
2099,1950-01-01,3039114625,2021-01-05,2,2020-12-31,Самовывоз,3PL 2,68503,Доставлен,Кластер Екатеринбург,Кластер Москва
3093,1950-01-01,304073092,2021-01-05,2,2021-01-08,Самовывоз,3PL 2,93233,Доставлен,Кластер Калининград,Кластер Краснодар
3290,1950-01-01,300232595,2021-01-05,2,2020-12-31,Самовывоз,3PL 2,68503,Доставлен,Кластер Новосибирск,Кластер Москва
5138,1950-01-01,3399142241,2021-03-22,2,1950-01-01,Самовывоз,3PL 2,833469,На пути к клиенту,Кластер Московская область,Кластер Екатеринбург
5279,1950-01-01,21414577700,2021-01-31,2,2021-02-01,Курьер,Самодоставка,120629,Доставлен,Кластер Москва,Кластер Екатеринбург
6180,1950-01-01,302073096,2021-01-05,2,2021-01-05,Самовывоз,3PL 2,833469,Доставлен,Кластер Московская область,Кластер Екатеринбург
6359,1950-01-01,302459167,2021-01-05,2,2021-01-05,Самовывоз,3PL 2,923125,Доставлен,Кластер Центр,Кластер Новосибирск


Вижу, что timeslot_number=2 при том, что запись о заказе всего одна и та без даты заказа. Удалю такие строки

In [130]:
logistics_online=logistics_online.drop(labels = [2060, 2099, 3093, 3290, 5138, 5279, 6180, 6359],axis =0)

In [131]:
logistics_online

,date_order,number_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster
0,2021-01-05,313655114,2021-01-14,1,2021-01-11,Самовывоз,3PL 1,301202,Доставлен,Кластер Краснодар,Кластер Москва
1,2021-01-03,315229153,2021-01-11,1,2021-01-08,Курьер,Самодоставка,40560,Доставлен,Кластер Московская область,Кластер Московская область
2,2021-01-08,3114106095,2021-01-16,1,2021-01-14,Курьер,Самодоставка,40560,Доставлен,Кластер Московская область,Кластер Московская область
3,2021-01-12,31145541407,2021-01-20,1,2021-01-15,Самовывоз,3PL 1,301202,Доставлен,Кластер Санкт-Петербург,Кластер Москва
4,2021-01-16,325050136,2021-01-26,1,2021-01-25,Самовывоз,3PL 1,946233,Доставлен,Кластер Санкт-Петербург,Кластер Санкт-Петербург
...,...,...,...,...,...,...,...,...,...,...,...
6592,2021-03-14,361544120,2021-03-19,1,2021-03-15,Самовывоз,3PL 2,64865,Доставлен,Кластер Краснодар,Кластер Северо-Запад
6593,2021-03-18,362569711,2021-03-22,1,1950-01-01,Самовывоз,3PL 2,64865,На пути к клиенту,Кластер Екатеринбург,Кластер Северо-Запад
6594,2021-03-14,3645314759,2021-03-23,1,1950-01-01,Самовывоз,3PL 2,9312933,На пути к клиенту,Кластер Краснодар,Кластер Казань
6595,2021-03-24,3649144391,2021-03-28,1,1950-01-01,Курьер,Самодоставка,902912,Ожидает сборки,Кластер Екатеринбург,Кластер Екатеринбург


- Проверю какие статусы у заказов, где нет даты доставки

In [132]:
logistics_online.query("result_data_order=='1950-01-01'")['order_status'].unique()

array(['Отменен', 'На пути к клиенту', 'Ожидает сборки'], dtype=object)

Здесь не наблюдаю несоответсвия

- Далее проверю на наличие ошибок в записях о Дате заказа и Дате доставки. Посмотрю есть ли заказы, где дата заказа позже даты доставки (смотрю только те значения, в которых указана дата доставки)

In [133]:
check=logistics_online.query('result_data_order<date_order')\
                        .query("result_data_order!='1950-01-01'").sort_values('date_order')
check.head()

,date_order,number_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster
6360,2020-12-28,305731992,2021-01-07,1,2020-12-23,Курьер,Самодоставка,833469,Доставлен,Кластер Казань,Кластер Екатеринбург
2570,2020-12-30,3031410942,2021-01-05,1,2020-12-29,Самовывоз,3PL 2,64865,Доставлен,Кластер Краснодар,Кластер Северо-Запад
4946,2020-12-31,30141433133,2021-01-11,1,2020-12-30,Курьер,Самодоставка,40560,Доставлен,Кластер Москва,Кластер Московская область
5645,2020-12-31,306160263,2021-01-08,1,2020-12-30,Курьер,Самодоставка,833469,Доставлен,Кластер Екатеринбург,Кластер Екатеринбург
1333,2021-01-01,311096952,2021-01-12,1,2020-12-31,Курьер,3PL 3,19,Доставлен,Кластер Центр,Кластер Москва


Видим наличие таких строк, в которых Дата заказа раньше, чем дата доставки. Такие данные считаю ошибочными и не буду использовать в расчете метрики. При этом, предполагаю, что среди них могут быть такие заказы, у которых timeslot_number	> 1. Проверю, так ли это

In [134]:
check=check[['date_order','number_order']]
check[['duplicated_nr']]=check[['number_order']] # столбец сномером заказа скопирую в новый столбец, как контрольный маркер
check

,date_order,number_order,duplicated_nr
6360,2020-12-28,305731992,305731992
2570,2020-12-30,3031410942,3031410942
4946,2020-12-31,30141433133,30141433133
5645,2020-12-31,306160263,306160263
1333,2021-01-01,311096952,311096952
...,...,...,...
5627,2021-03-27,346766153,346766153
3727,2021-04-01,3494931462,3494931462
2171,2021-04-02,3522714690,3522714690
2653,2021-04-05,352137569,352137569


In [135]:
logistics_online_check=logistics_online.query('result_data_order>date_order').sort_values('date_order') 
logistics_online_check.head()
#отбираю заказы, где дата формирования заказа < даты доставки

,date_order,number_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster
3294,1950-01-01,315759635,2021-01-23,2,2021-01-16,Самовывоз,3PL 1,301202,Доставлен,Кластер Центр,Кластер Москва
3248,1950-01-01,323166961,2021-01-26,2,2021-01-25,Самовывоз,3PL 2,34266,Доставлен,Кластер Краснодар,Кластер Москва
3244,1950-01-01,315531603,2021-01-23,2,2021-01-17,Самовывоз,3PL 1,301202,Доставлен,Кластер Центр,Кластер Москва
3236,1950-01-01,3603714277,2021-03-15,2,2021-03-15,Курьер,3PL 3,19923,Доставлен,Кластер Краснодар,Кластер Московская область
3202,1950-01-01,33695145314,2021-02-15,2,2021-02-13,Курьер,3PL 3,19923,Доставлен,Кластер Краснодар,Кластер Московская область


In [136]:
check=check.merge(logistics_online_check, on='number_order', how='right')
check.query("duplicated_nr!='NaN'").sort_values('duplicated_nr').head(20)
#Соединяю для проверки

,date_order_x,number_order,duplicated_nr,date_order_y,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster
589,2021-02-28,341450152,"341,450,152.00",1950-01-01,2021-03-11,2,2021-02-26,Самовывоз,3PL 2,19923,Доставлен,Кластер Новосибирск,Кластер Московская область
824,2021-03-05,344415660,"344,415,660.00",1950-01-01,2021-03-10,2,2021-02-24,Самовывоз,3PL 2,19923,Доставлен,Кластер Центр,Кластер Московская область
871,2021-03-09,347007292,"347,007,292.00",1950-01-01,2021-03-17,2,2021-03-02,Самовывоз,3PL 2,19923,Доставлен,Кластер Центр,Кластер Московская область
639,2021-03-11,349177995,"349,177,995.00",1950-01-01,2021-03-17,2,2021-03-01,Самовывоз,3PL 2,19923,Доставлен,Кластер Санкт-Петербург,Кластер Московская область
137,2021-01-16,3114694121,"3,114,694,121.00",1950-01-01,2021-01-22,2,2021-01-15,Курьер,3PL 1,5666,Доставлен,Кластер Санкт-Петербург,Кластер Санкт-Петербург
739,2021-02-26,3415371434,"3,415,371,434.00",1950-01-01,2021-03-06,2,2021-02-24,Самовывоз,3PL 2,19923,Доставлен,Кластер Екатеринбург,Кластер Московская область
672,2021-02-28,3417059914,"3,417,059,914.00",1950-01-01,2021-03-09,2,2021-02-19,Самовывоз,3PL 2,19923,Доставлен,Кластер Центр,Кластер Московская область
400,2021-02-24,3420147576,"3,420,147,576.00",1950-01-01,2021-03-02,2,2021-02-22,Самовывоз,3PL 2,19923,Доставлен,Кластер Московская область,Кластер Московская область
198,2021-03-04,34147014627,"34,147,014,627.00",1950-01-01,2021-03-13,2,2021-03-02,Самовывоз,3PL 2,93233,Доставлен,Кластер Краснодар,Кластер Краснодар


Обнаружилось 9 заказов, где дата доставки < даты заказа и количестов таймслотов > 1. Их нужно удалить из дф

In [137]:
logistics_online.query("number_order=='341450152' \
                        or number_order=='344415660' or number_order=='347007292' \
                        or number_order=='349177995' or number_order=='3114694121'\
                        or number_order=='3415371434' or number_order=='3417059914'\
                        or number_order=='3417059914' or number_order=='3420147576'\
                        or number_order=='34147014627'")

,date_order,number_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster
435,2021-03-04,34147014627,2021-03-12,1,2021-03-02,Самовывоз,3PL 2,93233,Доставлен,Кластер Краснодар,Кластер Краснодар
436,1950-01-01,34147014627,2021-03-13,2,2021-03-02,Самовывоз,3PL 2,93233,Доставлен,Кластер Краснодар,Кластер Краснодар
1212,2021-02-24,3420147576,2021-03-01,1,2021-02-22,Самовывоз,3PL 2,19923,Доставлен,Кластер Московская область,Кластер Московская область
1213,1950-01-01,3420147576,2021-03-02,2,2021-02-22,Самовывоз,3PL 2,19923,Доставлен,Кластер Московская область,Кластер Московская область
3511,2021-01-16,3114694121,2021-01-21,1,2021-01-15,Курьер,3PL 1,5666,Доставлен,Кластер Санкт-Петербург,Кластер Санкт-Петербург
3512,1950-01-01,3114694121,2021-01-22,2,2021-01-15,Курьер,3PL 1,5666,Доставлен,Кластер Санкт-Петербург,Кластер Санкт-Петербург
4270,2021-03-09,347007292,2021-03-16,1,2021-03-02,Самовывоз,3PL 2,19923,Доставлен,Кластер Центр,Кластер Московская область
4271,1950-01-01,347007292,2021-03-17,2,2021-03-02,Самовывоз,3PL 2,19923,Доставлен,Кластер Центр,Кластер Московская область
4576,1950-01-01,344415660,2021-03-10,2,2021-02-24,Самовывоз,3PL 2,19923,Доставлен,Кластер Центр,Кластер Московская область
4577,2021-03-05,344415660,2021-03-09,1,2021-02-24,Самовывоз,3PL 2,19923,Доставлен,Кластер Центр,Кластер Московская область


In [138]:
logistics_online=logistics_online.drop(labels = [435, 436, 1212, 1213, 3511, 3512, 4270, 4271, 4576, 4577, 5265, 5266, 5351, 5352, 5490, 5491, 5854, 5855],axis =0)

In [139]:
logistics_online=logistics_online.query('result_data_order>date_order').sort_values('date_order')
logistics_online

,date_order,number_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster
1107,1950-01-01,32634145140,2021-02-03,2,2021-02-03,Курьер,Самодоставка,6052,Доставлен,Кластер Екатеринбург,Кластер Москва
4321,1950-01-01,3574071495,2021-03-11,2,2021-03-11,Курьер,3PL 3,19923,Доставлен,Кластер Краснодар,Кластер Московская область
4318,1950-01-01,355193417,2021-03-11,2,2021-03-11,Курьер,3PL 1,5666,Доставлен,Кластер Краснодар,Кластер Санкт-Петербург
4306,1950-01-01,342167554,2021-02-24,2,2021-02-24,Самовывоз,3PL 2,51943,Доставлен,Кластер Москва,Кластер Краснодар
4304,1950-01-01,341500159,2021-02-25,2,2021-02-24,Курьер,3PL 2,3356412,Доставлен,Кластер Центр,Кластер Калининград
...,...,...,...,...,...,...,...,...,...,...,...
2281,2021-03-14,36261414753,2021-03-18,1,2021-03-15,Самовывоз,3PL 2,946233,Доставлен,Кластер Центр,Кластер Санкт-Петербург
4060,2021-03-14,36214531476,2021-03-23,1,2021-03-15,Самовывоз,3PL 1,833469,Доставлен,Кластер Казань,Кластер Екатеринбург
4688,2021-03-14,361570135,2021-03-19,1,2021-03-15,Самовывоз,3PL 2,51943,Доставлен,Кластер Казань,Кластер Краснодар
5783,2021-03-14,361350605,2021-03-18,1,2021-03-15,Самовывоз,3PL 1,301202,Доставлен,Кластер Краснодар,Кластер Москва


Найдем разницу между плановой и фактической датой доставки. Положительное значение разницы означает доставку раньше запланированного срока, отрицательное- позже

In [140]:
logistics_online['timedelta']=logistics_online['planned_data_order']-logistics_online['result_data_order']
logistics_online.sort_values('timedelta')

,date_order,number_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster,timedelta
3888,2021-01-25,3201442502,2021-01-29,1,2021-03-03,Курьер,3PL 2,4332332,Доставлен,Кластер Якутия,Кластер Казань,-33 days
6421,2021-01-27,33114014319,2021-02-03,1,2021-03-02,Самовывоз,3PL 1,86033,Доставлен,Кластер Москва,Кластер Москва,-27 days
6418,1950-01-01,33114014319,2021-02-04,2,2021-03-02,Самовывоз,3PL 1,86033,Доставлен,Кластер Москва,Кластер Москва,-26 days
3887,1950-01-01,3201442502,2021-02-10,2,2021-03-03,Курьер,3PL 2,4332332,Доставлен,Кластер Якутия,Кластер Казань,-21 days
6419,1950-01-01,33114014319,2021-02-10,3,2021-03-02,Самовывоз,3PL 1,86033,Доставлен,Кластер Москва,Кластер Москва,-20 days
...,...,...,...,...,...,...,...,...,...,...,...,...
2312,1950-01-01,3453026142,2021-03-11,2,2021-02-27,Самовывоз,3PL 1,833469,Доставлен,Кластер Казань,Кластер Екатеринбург,12 days
346,1950-01-01,3142551473,2021-02-02,2,2021-01-21,Самовывоз,3PL 2,43684,Доставлен,Кластер Екатеринбург,Кластер Казань,12 days
1145,1950-01-01,30361461496,2021-01-28,2,2021-01-11,Курьер,Самодоставка,833469,Доставлен,Кластер Московская область,Кластер Екатеринбург,17 days
3910,1950-01-01,3441446319,2021-03-24,3,2021-03-04,Курьер,Самодоставка,6052,Доставлен,Кластер Калининград,Кластер Москва,20 days


Создаю маркеры доставки в срок:  

In [141]:
logistics_online['timedelta'] = logistics_online['timedelta'].dt.days

In [142]:
logistics_online['on_time_status'] = (logistics_online['timedelta']>=0)
logistics_online.on_time_status = logistics_online.on_time_status.astype('int') # 0-не вовремя, 1-вовремя
logistics_online.head()

,date_order,number_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster,timedelta,on_time_status
1107,1950-01-01,32634145140,2021-02-03,2,2021-02-03,Курьер,Самодоставка,6052,Доставлен,Кластер Екатеринбург,Кластер Москва,0,1
4321,1950-01-01,3574071495,2021-03-11,2,2021-03-11,Курьер,3PL 3,19923,Доставлен,Кластер Краснодар,Кластер Московская область,0,1
4318,1950-01-01,355193417,2021-03-11,2,2021-03-11,Курьер,3PL 1,5666,Доставлен,Кластер Краснодар,Кластер Санкт-Петербург,0,1
4306,1950-01-01,342167554,2021-02-24,2,2021-02-24,Самовывоз,3PL 2,51943,Доставлен,Кластер Москва,Кластер Краснодар,0,1
4304,1950-01-01,341500159,2021-02-25,2,2021-02-24,Курьер,3PL 2,3356412,Доставлен,Кластер Центр,Кластер Калининград,1,1


1. Группировка по кол-ву попыток доставить в плановую дату доставки "timeslot_number"

В случае, когда доставка идет не с первого раза, в столбце с ожидаемой датой доставки указывается новая дата, при этом фактическая дата доставки по всем таймслотам будет единой. Т.о получается, что только последний слот доставки может быть отражен как доставка в плановую дату, а все предыдущие не своевременные. 

In [143]:
on_time_timeslot=logistics_online.groupby(['timeslot_number','on_time_status'], as_index=False)\
                            .agg({'number_order':'count'}).rename(columns={'number_order':'orders_count'})\
                            .pivot(index='timeslot_number', columns='on_time_status', values='orders_count')
on_time_timeslot['in_time_in_%']=on_time_timeslot[1]*100/(on_time_timeslot[0]+on_time_timeslot[1])
on_time_timeslot['in_time_in_%']=on_time_timeslot['in_time_in_%'].round(2)
on_time_timeslot

on_time_status,0,1,in_time_in_%
timeslot_number,,,
1,710.00,"3,300.00",82.29
2,86.00,369.00,81.10
3,19.00,45.00,70.31
4,2.00,13.00,86.67
5,nan,1.00,nan


Заказы с несколькими таймслотами отображаются в таблице несколько раз, соответственно. Исходя из рассуждений выше, если оставлять для анализа заказы с timeslot_number=N в N-ном количестве строк, то это будет искажать данные о количестве заказов и своевременности доставки. Поэтому, для таких заказов оставляю только одну строку

In [144]:
on_time=logistics_online.sort_values(['number_order', 'date_order'], ascending=False)\
                        .drop_duplicates(subset='number_order', keep='first')
on_time.sort_values('date_order').head()

,date_order,number_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster,timedelta,on_time_status
1918,2020-12-21,303621239,2021-01-01,1,2020-12-25,Самовывоз,3PL 2,68503,Доставлен,Кластер Казань,Кластер Москва,7,1
4231,2020-12-22,305403721,2021-01-02,1,2021-01-05,Самовывоз,3PL 2,33120125,Доставлен,Кластер Северо-Запад,Кластер Краснодар,-3,0
6358,2020-12-23,30150141149,2021-01-01,1,2020-12-29,Самовывоз,3PL 1,2619,Доставлен,Кластер Екатеринбург,Кластер Москва,3,1
2718,2020-12-24,30665014144,2021-01-01,1,2021-01-04,Самовывоз,3PL 2,989330,Доставлен,Кластер Краснодар,Кластер Екатеринбург,-3,0
1509,2020-12-24,307256994,2021-01-01,1,2021-01-04,Самовывоз,3PL 2,68503,Доставлен,Кластер Екатеринбург,Кластер Москва,-3,0


In [145]:
on_time.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4010 entries, 3774 to 6460
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   date_order          4010 non-null   datetime64[ns]
 1   number_order        4010 non-null   int64         
 2   planned_data_order  4010 non-null   datetime64[ns]
 3   timeslot_number     4010 non-null   int8          
 4   result_data_order   4010 non-null   datetime64[ns]
 5   delivery_type       4010 non-null   object        
 6   carrier_name        4010 non-null   object        
 7   seller_ID           4010 non-null   int64         
 8   order_status        4010 non-null   object        
 9   customer_cluster    4010 non-null   object        
 10  warehouse_cluster   4010 non-null   object        
 11  timedelta           4010 non-null   int64         
 12  on_time_status      4010 non-null   int64         
dtypes: datetime64[ns](3), int64(4), int8(1), obje

In [146]:
on_time.to_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-s-androsova-23/Final_project_рабочая/on_time.csv')
# выгужу файл для построения графиков в Табло

Визуализация в Tableau 

https://public.tableau.com/authoring/Project_Androsova_2022/Ozonmetrcicsdashboard_1#1

2. Посчитать PromisedClick2Delivery - метрика обещанного срока доставки (от даты заказа до первой плановой даты доставки), в сутках.

 - Необходима динамика по дням, неделям, месяцам;

 - Необходимо сравнение перевозчиков;

 - Необходимо понимание уровня метрики из кластера в кластер.

Воспользуюсь полученным ранее дф logistics_online и применю фильтр timeslot_number=1. Найду разницу между плановой датой доставки и датой заказа

In [147]:
logistics_online_promised=logistics_online.query("timeslot_number=='1'")
logistics_online_promised['PromisedClick2Delivery']=logistics_online_promised['planned_data_order']\
                                                   - logistics_online_promised['date_order']
logistics_online_promised['PromisedClick2Delivery'] = logistics_online_promised['PromisedClick2Delivery'].dt.days
logistics_online_promised.sort_values('PromisedClick2Delivery').head()

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,date_order,number_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster,timedelta,on_time_status,PromisedClick2Delivery
2591,2021-02-16,3446491405,2021-02-20,1,2021-02-20,Курьер,Самодоставка,9441212,Доставлен,Кластер Москва,Кластер Москва,0,1,4
5301,2021-02-15,3420140912,2021-02-19,1,2021-02-17,Самовывоз,3PL 1,301202,Доставлен,Кластер Казань,Кластер Москва,2,1,4
3311,2021-02-15,339213430,2021-02-19,1,2021-02-18,Самовывоз,3PL 2,51943,Доставлен,Кластер Казань,Кластер Краснодар,1,1,4
5951,2021-02-15,3432471464,2021-02-19,1,2021-02-20,Самовывоз,3PL 2,19923,Доставлен,Кластер Казань,Кластер Московская область,-1,0,4
4147,2021-01-21,321074146,2021-01-25,1,2021-01-23,Самовывоз,3PL 2,833469,Доставлен,Кластер Московская область,Кластер Екатеринбург,2,1,4


In [148]:
logistics_online_promised.to_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-s-androsova-23/Final_project_рабочая/PromisedClick2Delivery.csv')
# выгужу файл для построения графиков в Табло

In [149]:
clusters_map_PromisedClick2Delivery=logistics_online_promised.groupby(['warehouse_cluster', 'customer_cluster'], as_index = False)\
                    .agg({'PromisedClick2Delivery' : 'mean'}).rename(columns={'PromisedClick2Delivery':'PromisedClick2Delivery'}).round(1)
clusters_map_PromisedClick2Delivery

,warehouse_cluster,customer_cluster,PromisedClick2Delivery
0,Кластер Екатеринбург,Кластер Екатеринбург,9.10
1,Кластер Екатеринбург,Кластер Казань,8.50
2,Кластер Екатеринбург,Кластер Калининград,7.50
3,Кластер Екатеринбург,Кластер Краснодар,8.10
4,Кластер Екатеринбург,Кластер Москва,7.60
...,...,...,...
108,Кластер Центр,Кластер Московская область,8.60
109,Кластер Центр,Кластер Новосибирск,9.40
110,Кластер Центр,Кластер Санкт-Петербург,7.40
111,Кластер Центр,Кластер Северо-Запад,8.30


In [150]:
PromisedClick2Delivery_clusters_map=clusters_map_PromisedClick2Delivery\
                            .pivot(index ='warehouse_cluster', columns='customer_cluster', values='PromisedClick2Delivery')\
                            .replace(np.nan, '')

In [151]:
PromisedClick2Delivery_clusters_map

customer_cluster,Кластер Екатеринбург,Кластер Казань,Кластер Калининград,Кластер Краснодар,Кластер Москва,Кластер Московская область,Кластер Новосибирск,Кластер Санкт-Петербург,Кластер Северо-Запад,Кластер Хабаровск,Кластер Центр,Кластер Якутия
warehouse_cluster,,,,,,,,,,,,
Кластер Екатеринбург,9.10,8.50,7.50,8.10,7.60,7.10,8.20,7.00,8.90,10.00,8.50,8.50
Кластер Казань,7.70,7.80,9.70,8.10,7.80,7.50,7.60,7.70,8.20,10.50,7.80,4.00
Кластер Калининград,6.90,8.30,,6.20,7.60,8.30,9.80,8.00,6.80,,9.20,
Кластер Краснодар,8.10,7.70,7.00,7.60,7.60,7.80,7.30,7.80,8.10,9.00,8.20,6.00
Кластер Москва,7.70,7.80,6.00,7.70,7.50,7.70,8.30,7.50,8.10,7.50,7.60,
Кластер Московская область,8.00,7.70,,7.40,7.70,7.70,7.40,7.40,8.30,8.20,7.80,6.00
Кластер Новосибирск,7.70,8.70,5.00,8.50,8.00,8.50,7.70,9.30,9.30,9.00,8.10,8.00
Кластер Санкт-Петербург,7.90,7.70,7.50,7.90,7.70,7.60,8.00,7.60,6.20,,8.20,
Кластер Северо-Запад,7.20,7.40,7.70,7.60,7.20,7.40,9.70,7.80,8.70,7.00,7.80,


3. Посчитать Click2Delivery - метрика фактического срока доставки (от даты заказа до момента вручения), в сутках.

 - Необходимо сравнение с метрикой PromisedClick2Delivery;

 - Необходимо понимание кластеров, где заказы доставляются быстрее, чем мы обещаем клиентам;

In [152]:
logistics_online_promised['Click2Delivery']=logistics_online_promised['result_data_order']-logistics_online_promised['date_order']
logistics_online_promised['Click2Delivery']=logistics_online_promised['Click2Delivery'].dt.days

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [153]:
logistics_online_promised.sort_values('PromisedClick2Delivery').head()

,date_order,number_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster,timedelta,on_time_status,PromisedClick2Delivery,Click2Delivery
2591,2021-02-16,3446491405,2021-02-20,1,2021-02-20,Курьер,Самодоставка,9441212,Доставлен,Кластер Москва,Кластер Москва,0,1,4,4
5301,2021-02-15,3420140912,2021-02-19,1,2021-02-17,Самовывоз,3PL 1,301202,Доставлен,Кластер Казань,Кластер Москва,2,1,4,2
3311,2021-02-15,339213430,2021-02-19,1,2021-02-18,Самовывоз,3PL 2,51943,Доставлен,Кластер Казань,Кластер Краснодар,1,1,4,3
5951,2021-02-15,3432471464,2021-02-19,1,2021-02-20,Самовывоз,3PL 2,19923,Доставлен,Кластер Казань,Кластер Московская область,-1,0,4,5
4147,2021-01-21,321074146,2021-01-25,1,2021-01-23,Самовывоз,3PL 2,833469,Доставлен,Кластер Московская область,Кластер Екатеринбург,2,1,4,2


In [154]:
logistics_online_promised.to_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-s-androsova-23/Final_project_рабочая/Click2Delivery.csv')
# выгужу файл для построения графиков в Табло

Покажу метрику Click2Delivery в зависимости от кластера - склада и покупателя

In [155]:
clusters_map_Click2Delivery=logistics_online_promised.groupby(['warehouse_cluster', 'customer_cluster'], as_index = False)\
                    .agg({'Click2Delivery' : 'mean'}).round(1)
clusters_map_Click2Delivery

,warehouse_cluster,customer_cluster,Click2Delivery
0,Кластер Екатеринбург,Кластер Екатеринбург,3.10
1,Кластер Екатеринбург,Кластер Казань,4.20
2,Кластер Екатеринбург,Кластер Калининград,4.50
3,Кластер Екатеринбург,Кластер Краснодар,4.50
4,Кластер Екатеринбург,Кластер Москва,5.60
...,...,...,...
108,Кластер Центр,Кластер Московская область,5.40
109,Кластер Центр,Кластер Новосибирск,4.40
110,Кластер Центр,Кластер Санкт-Петербург,4.70
111,Кластер Центр,Кластер Северо-Запад,7.00


In [156]:
Click2Delivery_clusters_map=clusters_map_Click2Delivery\
                .pivot(index ='warehouse_cluster', columns='customer_cluster', values='Click2Delivery')\
                .replace(np.nan, '')
Click2Delivery_clusters_map

customer_cluster,Кластер Екатеринбург,Кластер Казань,Кластер Калининград,Кластер Краснодар,Кластер Москва,Кластер Московская область,Кластер Новосибирск,Кластер Санкт-Петербург,Кластер Северо-Запад,Кластер Хабаровск,Кластер Центр,Кластер Якутия
warehouse_cluster,,,,,,,,,,,,
Кластер Екатеринбург,3.10,4.20,4.50,4.50,5.60,5.40,5.20,4.50,4.90,6.00,4.50,8.50
Кластер Казань,4.80,5.20,9.00,5.70,6.10,5.70,5.70,5.00,5.10,9.50,5.40,22.00
Кластер Калининград,5.50,7.10,,5.00,7.50,7.90,7.50,5.30,7.40,,7.90,
Кластер Краснодар,5.10,6.30,4.00,5.80,6.90,6.00,4.70,5.60,6.50,9.40,5.60,12.50
Кластер Москва,6.80,7.00,10.20,7.90,7.30,7.50,7.80,5.70,6.80,12.80,6.70,
Кластер Московская область,6.40,6.60,,7.10,7.70,7.00,6.20,5.90,7.00,9.20,6.70,4.70
Кластер Новосибирск,4.90,4.60,4.00,8.80,5.20,4.10,6.10,5.00,6.80,6.00,3.90,4.00
Кластер Санкт-Петербург,5.40,5.70,15.50,6.30,6.20,4.90,6.60,6.30,3.90,,6.00,
Кластер Северо-Запад,4.40,4.60,6.70,3.80,6.60,7.40,9.30,5.50,5.80,3.00,7.60,


Сравню между собой PromisedClick2Delivery и Click2Delivery по каждому кластеру

In [157]:
compare=clusters_map_Click2Delivery.merge(clusters_map_PromisedClick2Delivery, on=['customer_cluster', 'warehouse_cluster'])
compare['difference']=compare['PromisedClick2Delivery']-compare['Click2Delivery']
compare

,warehouse_cluster,customer_cluster,Click2Delivery,PromisedClick2Delivery,difference
0,Кластер Екатеринбург,Кластер Екатеринбург,3.10,9.10,6.00
1,Кластер Екатеринбург,Кластер Казань,4.20,8.50,4.30
2,Кластер Екатеринбург,Кластер Калининград,4.50,7.50,3.00
3,Кластер Екатеринбург,Кластер Краснодар,4.50,8.10,3.60
4,Кластер Екатеринбург,Кластер Москва,5.60,7.60,2.00
...,...,...,...,...,...
108,Кластер Центр,Кластер Московская область,5.40,8.60,3.20
109,Кластер Центр,Кластер Новосибирск,4.40,9.40,5.00
110,Кластер Центр,Кластер Санкт-Петербург,4.70,7.40,2.70
111,Кластер Центр,Кластер Северо-Запад,7.00,8.30,1.30


In [158]:
compare_map=compare[compare.difference>0].pivot(index ='warehouse_cluster', columns='customer_cluster', values='difference').replace(np.nan, '')
compare_map

customer_cluster,Кластер Екатеринбург,Кластер Казань,Кластер Калининград,Кластер Краснодар,Кластер Москва,Кластер Московская область,Кластер Новосибирск,Кластер Санкт-Петербург,Кластер Северо-Запад,Кластер Хабаровск,Кластер Центр,Кластер Якутия
warehouse_cluster,,,,,,,,,,,,
Кластер Екатеринбург,6.00,4.30,3.00,3.60,2.00,1.70,3.00,2.50,4.00,4.00,4.00,
Кластер Казань,2.90,2.60,0.70,2.40,1.70,1.80,1.90,2.70,3.10,1.00,2.40,
Кластер Калининград,1.40,1.20,,1.20,0.10,0.40,2.30,2.70,,,1.30,
Кластер Краснодар,3.00,1.40,3.00,1.80,0.70,1.80,2.60,2.20,1.60,,2.60,
Кластер Москва,0.90,0.80,,,0.20,0.20,0.50,1.80,1.30,,0.90,
Кластер Московская область,1.60,1.10,,0.30,,0.70,1.20,1.50,1.30,,1.10,1.30
Кластер Новосибирск,2.80,4.10,1.00,,2.80,4.40,1.60,4.30,2.50,3.00,4.20,4.00
Кластер Санкт-Петербург,2.50,2.00,,1.60,1.50,2.70,1.40,1.30,2.30,,2.20,
Кластер Северо-Запад,2.80,2.80,1.00,3.80,0.60,,0.40,2.30,2.90,4.00,0.20,


4. Посчитать GMV Accepted - метрика совокупного объема продаж (считается по заказам, оформленным в указанный день), в рублях.

 - Необходимо видеть изменение метрики по дням;

 - Необходимы разрезы по перевозчикам, по типу доставки, по кластерам склада;

In [159]:
product_online = '''
    SELECT 
        number_order,
        product_ID,
        product_price,
        amount 
    FROM default.product_online 
'''

product_online = ph.read_clickhouse(query = product_online, connection = connection_default)
product_online.head()

,number_order,product_ID,product_price,amount
0,313655114,983587801,"5,512.44",1
1,313655114,983587158,"4,070.35",2
2,313655114,983582669,"3,591.97",1
3,315229153,983589424,"4,433.23",3
4,315229153,983587827,"2,862.50",2


In [160]:
product_online.head()

,number_order,product_ID,product_price,amount
0,313655114,983587801,"5,512.44",1
1,313655114,983587158,"4,070.35",2
2,313655114,983582669,"3,591.97",1
3,315229153,983589424,"4,433.23",3
4,315229153,983587827,"2,862.50",2


In [161]:
product_online.nunique()

number_order     5976
product_ID       7057
product_price    7031
amount              4
dtype: int64

In [162]:
product_online.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12233 entries, 0 to 12232
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   number_order   12233 non-null  int64  
 1   product_ID     12233 non-null  int64  
 2   product_price  12233 non-null  float64
 3   amount         12233 non-null  int16  
dtypes: float64(1), int16(1), int64(2)
memory usage: 310.7 KB


Считаю сумму каждого заказа

In [163]:
GMV_Accepted=product_online [['number_order','product_ID', 'product_price', 'amount']] 
#перезапишу дф в GMV Accepted

In [164]:
GMV_Accepted['GMV_Accepted']=GMV_Accepted.product_price * GMV_Accepted.amount

In [165]:
GMV_Accepted

,number_order,product_ID,product_price,amount,GMV_Accepted
0,313655114,983587801,"5,512.44",1,"5,512.44"
1,313655114,983587158,"4,070.35",2,"8,140.70"
2,313655114,983582669,"3,591.97",1,"3,591.97"
3,315229153,983589424,"4,433.23",3,"13,299.69"
4,315229153,983587827,"2,862.50",2,"5,725.00"
...,...,...,...,...,...
12228,3649144391,983583847,"8,146.30",2,"16,292.60"
12229,3649144391,983581209,"2,816.34",4,"11,265.36"
12230,365435447,983582430,"2,937.91",4,"11,751.64"
12231,365435447,983589903,"7,626.13",4,"30,504.52"


In [166]:
GMV_Accepted=GMV_Accepted.groupby('number_order', as_index=False).agg({'GMV_Accepted':'sum'})
GMV_Accepted.sort_values('GMV_Accepted')

,number_order,GMV_Accepted
720,333139132,27.25
5906,341461447147,86.99
2357,360141951,96.90
4630,3573361141,118.65
3118,3251439763,118.65
...,...,...
3590,3366677014,"127,825.52"
3499,3341471469,"130,702.67"
2270,359413954,"132,128.63"
2721,364563365,"155,562.69"


In [167]:
logistics_online_2=logistics_online.query("date_order!='1950-01-01'") #заказы с пустой датой заказа отсеиваю

Для получения информации о датах заказов, объединю с logistics_online_2

In [168]:
GMV_Accepted_merged=logistics_online_2.merge(GMV_Accepted, on='number_order')

In [169]:
GMV_Accepted_merged.sort_values('date_order')

,date_order,number_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster,timedelta,on_time_status,GMV_Accepted
0,2020-12-21,303621239,2021-01-01,1,2020-12-25,Самовывоз,3PL 2,68503,Доставлен,Кластер Казань,Кластер Москва,7,1,"24,288.67"
1,2020-12-22,305403721,2021-01-02,1,2021-01-05,Самовывоз,3PL 2,33120125,Доставлен,Кластер Северо-Запад,Кластер Краснодар,-3,0,"12,108.50"
2,2020-12-23,30150141149,2021-01-01,1,2020-12-29,Самовывоз,3PL 1,2619,Доставлен,Кластер Екатеринбург,Кластер Москва,3,1,"4,307.10"
3,2020-12-24,307256994,2021-01-01,1,2021-01-04,Самовывоз,3PL 2,68503,Доставлен,Кластер Екатеринбург,Кластер Москва,-3,0,"30,159.34"
4,2020-12-24,30665014144,2021-01-01,1,2021-01-04,Самовывоз,3PL 2,989330,Доставлен,Кластер Краснодар,Кластер Екатеринбург,-3,0,"2,789.18"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4008,2021-03-14,361350605,2021-03-18,1,2021-03-15,Самовывоз,3PL 1,301202,Доставлен,Кластер Краснодар,Кластер Москва,3,1,"17,454.51"
4005,2021-03-14,36261414753,2021-03-18,1,2021-03-15,Самовывоз,3PL 2,946233,Доставлен,Кластер Центр,Кластер Санкт-Петербург,3,1,"29,515.08"
4006,2021-03-14,36214531476,2021-03-23,1,2021-03-15,Самовывоз,3PL 1,833469,Доставлен,Кластер Казань,Кластер Екатеринбург,8,1,"6,007.42"
4007,2021-03-14,361570135,2021-03-19,1,2021-03-15,Самовывоз,3PL 2,51943,Доставлен,Кластер Казань,Кластер Краснодар,4,1,"40,646.76"


In [170]:
GMV_Accepted_merged.to_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-s-androsova-23/Final_project_рабочая/GMV_Accepted.csv')
# выгужу файл для построения графиков в Табло

In [171]:
returns_online = '''
SELECT 
    number_order,
    product_ID,
    amount_return,
    return_date
from  default.returns_online
'''

returns_online = ph.read_clickhouse(query = returns_online, connection = connection_default)
returns_online.sort_values('return_date').head(5)

,number_order,product_ID,amount_return,return_date
0,3031410942,983588011,1,1970-01-01
231,341075161,983581371,1,1970-01-01
230,3435959014,983580818,3,1970-01-01
229,3414466114,983589259,1,1970-01-01
228,3439661413,983585097,1,1970-01-01


In [172]:
returns_online.nunique()

number_order     282
product_ID       338
amount_return      4
return_date       28
dtype: int64

Изначально этого не видно, но впоследствии, при объединении таблиц становится понятно (разница между датой заказа и датой возврата очень большая, что кажется неправдоподобным), что дата в столбце return_date записана в формате гггг-дд-мм. Поэтому, нужно перевести в формат гггг-мм-дд

In [173]:
returns_online.return_date = returns_online.return_date.dt.strftime('%Y-%d-%m')
returns_online['return_date'] = pd.to_datetime(returns_online['return_date'], format="%Y-%m-%d", errors='coerce')

In [174]:
returns_online.sort_values('return_date')

,number_order,product_ID,amount_return,return_date
0,3031410942,983588011,1,1970-01-01
231,341075161,983581371,1,1970-01-01
230,3435959014,983580818,3,1970-01-01
229,3414466114,983589259,1,1970-01-01
228,3439661413,983585097,1,1970-01-01
...,...,...,...,...
415,361131249,983583035,1,2021-03-12
424,3595014033,983585636,1,2021-03-12
423,360565100,983582699,1,2021-03-12
426,360622343,983589205,1,2021-03-12


In [175]:
returns_online.query("number_order=='335147961' or number_order=='331157245'").sort_values('product_ID')
# беру эти заказы случайным образом из тех, что выдавались задублированными (ниже)

,number_order,product_ID,amount_return,return_date
151,335147961,983581409,1,2021-02-12
160,335147961,983581409,1,1970-01-01
149,335147961,983585789,1,2021-02-12
161,335147961,983585789,1,1970-01-01
150,335147961,983589307,1,2021-02-12
162,335147961,983589307,1,1970-01-01
129,331157245,983589479,3,2021-02-07
136,331157245,983589479,3,2021-02-09


из этих двух примеров заказов выше вижу что дублирование может быть как по датам, так и есть странная дата 1970. Поэтому удалю записи с датой 1970, а также все дублирующие номера заказов

In [176]:
returns_online.loc[returns_online.duplicated(['number_order', 'product_ID'])].sort_values('number_order')

,number_order,product_ID,amount_return,return_date
19,312396246,983582111,1,1970-01-01
22,312396246,983588915,1,1970-01-01
23,312396246,983582678,1,1970-01-01
24,312396246,983583606,1,1970-01-01
25,312396246,983580555,1,1970-01-01
...,...,...,...,...
493,35714135014,983587402,2,1970-01-01
495,35714135014,983584567,3,1970-01-01
456,35961419114,983583893,3,1970-01-01
457,35961419114,983589871,1,1970-01-01


In [177]:
returns_online.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496 entries, 0 to 495
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   number_order   496 non-null    int64         
 1   product_ID     496 non-null    int64         
 2   amount_return  496 non-null    int8          
 3   return_date    496 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(2), int8(1)
memory usage: 12.2 KB


In [178]:
returns_online=returns_online.sort_values(['return_date'], ascending=False)

In [179]:
returns_online=returns_online.drop_duplicates(['number_order', 'product_ID']).query("return_date!='1970-01-01'")
returns_online

,number_order,product_ID,amount_return,return_date
428,361364500,983589405,1,2021-03-12
414,355142634,983587499,1,2021-03-12
426,360622343,983589205,1,2021-03-12
425,359626936,983585419,1,2021-03-12
424,3595014033,983585636,1,2021-03-12
...,...,...,...,...
4,3131450699,983589729,3,2021-01-12
6,315774746,983585035,1,2021-01-12
2,3147145173,983589156,1,2021-01-11
3,3141464669,983589362,1,2021-01-11


In [180]:
Return_Rate_df=logistics_online_2.merge(returns_online, on='number_order', how='left')
Return_Rate_df.sort_values('return_date').head(5)

,date_order,number_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster,timedelta,on_time_status,product_ID,amount_return,return_date
42,2021-01-02,3147145173,2021-01-10,1,2021-01-08,Курьер,Самодоставка,40560,Доставлен,Кластер Московская область,Кластер Московская область,2,1,"983,589,156.00",1.00,2021-01-11
64,2021-01-03,3141464669,2021-01-13,1,2021-01-09,Самовывоз,3PL 2,41958,Доставлен,Кластер Центр,Кластер Казань,4,1,"983,589,362.00",1.00,2021-01-11
209,2021-01-09,315774746,2021-01-14,1,2021-01-11,Самовывоз,3PL 1,3125312,Доставлен,Кластер Московская область,Кластер Москва,3,1,"983,585,035.00",1.00,2021-01-12
580,2021-01-21,32331221414,2021-02-01,1,2021-01-30,Самовывоз,3PL 2,833469,Доставлен,Кластер Калининград,Кластер Екатеринбург,2,1,"983,587,528.00",1.00,2021-02-01
581,2021-01-21,3267153714,2021-01-28,1,2021-01-29,Самовывоз,3PL 1,30888,Доставлен,Кластер Екатеринбург,Кластер Москва,-1,0,"983,584,322.00",1.00,2021-02-01


считаю количество возвратов и заказов

- Необходимо видеть изменение метрики по дням;

- Необходимы разрезы по перевозчикам, по кластерам склада и клиента;

In [181]:
returned_orders = Return_Rate_df.groupby(['return_date','delivery_type', 'carrier_name', 'customer_cluster', 'warehouse_cluster'],  as_index = False)\
                                    .agg({'number_order' : 'count'})\
                                    .rename(columns = {'number_order' : 'returned_orders_count','return_date' : 'date'})
returned_orders.sort_values('date').head()

,date,delivery_type,carrier_name,customer_cluster,warehouse_cluster,returned_orders_count
0,2021-01-11,Курьер,Самодоставка,Кластер Московская область,Кластер Московская область,1
1,2021-01-11,Самовывоз,3PL 2,Кластер Центр,Кластер Казань,1
2,2021-01-12,Самовывоз,3PL 1,Кластер Московская область,Кластер Москва,1
3,2021-02-01,Самовывоз,3PL 1,Кластер Екатеринбург,Кластер Москва,1
4,2021-02-01,Самовывоз,3PL 1,Кластер Москва,Кластер Екатеринбург,1


In [182]:
orders=Return_Rate_df.groupby(['date_order', 'delivery_type', 'carrier_name', 'customer_cluster', 'warehouse_cluster'], as_index = False)\
                                    .agg({'number_order':'count'})\
                                    .rename(columns = {'number_order' : 'orders_count', 'date_order' : 'date'})
orders.head()

,date,delivery_type,carrier_name,customer_cluster,warehouse_cluster,orders_count
0,2020-12-21,Самовывоз,3PL 2,Кластер Казань,Кластер Москва,1
1,2020-12-22,Самовывоз,3PL 2,Кластер Северо-Запад,Кластер Краснодар,1
2,2020-12-23,Самовывоз,3PL 1,Кластер Екатеринбург,Кластер Москва,1
3,2020-12-24,Самовывоз,3PL 2,Кластер Екатеринбург,Кластер Москва,1
4,2020-12-24,Самовывоз,3PL 2,Кластер Краснодар,Кластер Екатеринбург,1


Исходя из задания ("доля возвратов (отношение количества возвратов к заказам в этот день)") необходимы данные только за те дни, в которые были возвраты. Однако если объединить таблицы по дф returned_orders, то при расчете метрики теряются данные заказов по несовпадающим перевозчикам и кластерам, и результат в разрезе только даты оказывается некорректым. Поэтому сделаю полное объединение таблиц

In [183]:
Return_rate=orders.merge(returned_orders, on = ['date', 'delivery_type', 'carrier_name', 'customer_cluster', 'warehouse_cluster'], how="outer")
Return_rate.sort_values('date').head()

,date,delivery_type,carrier_name,customer_cluster,warehouse_cluster,orders_count,returned_orders_count
0,2020-12-21,Самовывоз,3PL 2,Кластер Казань,Кластер Москва,1.00,nan
1,2020-12-22,Самовывоз,3PL 2,Кластер Северо-Запад,Кластер Краснодар,1.00,nan
2,2020-12-23,Самовывоз,3PL 1,Кластер Екатеринбург,Кластер Москва,1.00,nan
3,2020-12-24,Самовывоз,3PL 2,Кластер Екатеринбург,Кластер Москва,1.00,nan
4,2020-12-24,Самовывоз,3PL 2,Кластер Краснодар,Кластер Екатеринбург,1.00,nan


In [184]:
Return_rate.to_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-s-androsova-23/Final_project_рабочая/Return_rate.csv')
# выгужу файл для построения графиков в Табло

6. Посчитать GMV D-R - метрика, отражающая совокупный объем доставленных заказов за вычетом возвратов, в рублях.

 - Необходимо видеть изменение метрики по дням (для всех дней, которые были раньше, чем дата расчета метрики);

 - Необходимы разрезы по перевозчикам, по типу доставки, по кластерам склада.

Инфо о стоимости продуктов в дф product_online:

In [185]:
product_online.head()

,number_order,product_ID,product_price,amount
0,313655114,983587801,"5,512.44",1
1,313655114,983587158,"4,070.35",2
2,313655114,983582669,"3,591.97",1
3,315229153,983589424,"4,433.23",3
4,315229153,983587827,"2,862.50",2


Доходы от продажи товаров посчитаны ранее:

In [186]:
GMV_Accepted_merged.sort_values('date_order').head() #выручка

,date_order,number_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster,timedelta,on_time_status,GMV_Accepted
0,2020-12-21,303621239,2021-01-01,1,2020-12-25,Самовывоз,3PL 2,68503,Доставлен,Кластер Казань,Кластер Москва,7,1,"24,288.67"
1,2020-12-22,305403721,2021-01-02,1,2021-01-05,Самовывоз,3PL 2,33120125,Доставлен,Кластер Северо-Запад,Кластер Краснодар,-3,0,"12,108.50"
2,2020-12-23,30150141149,2021-01-01,1,2020-12-29,Самовывоз,3PL 1,2619,Доставлен,Кластер Екатеринбург,Кластер Москва,3,1,"4,307.10"
3,2020-12-24,307256994,2021-01-01,1,2021-01-04,Самовывоз,3PL 2,68503,Доставлен,Кластер Екатеринбург,Кластер Москва,-3,0,"30,159.34"
4,2020-12-24,30665014144,2021-01-01,1,2021-01-04,Самовывоз,3PL 2,989330,Доставлен,Кластер Краснодар,Кластер Екатеринбург,-3,0,"2,789.18"


Возвраты :

In [187]:
Return_Rate_df.query("number_order=='33171914714'").sort_values(['return_date', 'number_order']).head()

,date_order,number_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster,timedelta,on_time_status,product_ID,amount_return,return_date
841,2021-01-27,33171914714,2021-02-05,1,2021-02-03,Курьер,3PL 2,41958,Доставлен,Кластер Москва,Кластер Казань,2,1,"983,584,499.00",1.00,2021-02-06
842,2021-01-27,33171914714,2021-02-05,1,2021-02-03,Курьер,3PL 2,41958,Доставлен,Кластер Москва,Кластер Казань,2,1,"983,588,995.00",2.00,2021-02-06
843,2021-01-27,33171914714,2021-02-05,1,2021-02-03,Курьер,3PL 2,41958,Доставлен,Кластер Москва,Кластер Казань,2,1,"983,586,977.00",2.00,2021-02-06


In [188]:
Return_Rate_df.nunique()

date_order              84
number_order          4010
planned_data_order      79
timeslot_number          1
result_data_order       76
delivery_type            2
carrier_name             5
seller_ID              143
order_status             3
customer_cluster        12
warehouse_cluster       11
timedelta               27
on_time_status           2
product_ID             135
amount_return            4
return_date             26
dtype: int64

Объединю Return_Rate_df и product_online что бы получить стоимость возвратов

In [189]:
Return_loss=Return_Rate_df.merge(product_online, on=['number_order', 'product_ID'], how='left')

In [190]:
Return_loss['loss_from_return']=Return_loss['amount_return']*Return_loss['product_price']
Return_loss.sort_values('loss_from_return')

,date_order,number_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster,timedelta,on_time_status,product_ID,amount_return,return_date,product_price,amount,loss_from_return
3645,2021-03-05,359019171,2021-03-10,1,2021-03-11,Курьер,Самодоставка,9441212,Доставлен,Кластер Москва,Кластер Москва,-1,0,"983,581,342.00",1.00,2021-03-12,761.81,2.00,761.81
3492,2021-03-04,355142634,2021-03-15,1,2021-03-10,Курьер,3PL 1,5620,Доставлен,Кластер Казань,Кластер Казань,5,1,"983,587,499.00",1.00,2021-03-12,848.47,1.00,848.47
1263,2021-02-04,33271441490,2021-02-09,1,2021-02-06,Самовывоз,3PL 1,334424,Доставлен,Кластер Центр,Кластер Краснодар,3,1,"983,585,410.00",1.00,2021-02-07,861.36,2.00,861.36
2882,2021-02-25,354011302,2021-03-06,1,2021-03-11,Курьер,Самодоставка,681242,Доставлен,Кластер Москва,Кластер Москва,-5,0,"983,584,286.00",1.00,2021-03-12,"1,243.10",1.00,"1,243.10"
1616,2021-02-09,335147961,2021-02-16,1,2021-02-11,Самовывоз,3PL 1,49633,Доставлен,Кластер Центр,Кластер Новосибирск,5,1,"983,589,307.00",1.00,2021-02-12,"1,783.49",1.00,"1,783.49"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4030,2021-03-14,36261414753,2021-03-18,1,2021-03-15,Самовывоз,3PL 2,946233,Доставлен,Кластер Центр,Кластер Санкт-Петербург,3,1,nan,nan,NaT,nan,nan,nan
4031,2021-03-14,36214531476,2021-03-23,1,2021-03-15,Самовывоз,3PL 1,833469,Доставлен,Кластер Казань,Кластер Екатеринбург,8,1,nan,nan,NaT,nan,nan,nan
4032,2021-03-14,361570135,2021-03-19,1,2021-03-15,Самовывоз,3PL 2,51943,Доставлен,Кластер Казань,Кластер Краснодар,4,1,nan,nan,NaT,nan,nan,nan
4033,2021-03-14,361350605,2021-03-18,1,2021-03-15,Самовывоз,3PL 1,301202,Доставлен,Кластер Краснодар,Кластер Москва,3,1,nan,nan,NaT,nan,nan,nan


Считаю сумму возврата по каждому заказу

In [191]:
Return_loss=Return_loss.groupby(['date_order', 'number_order', 'carrier_name', 'delivery_type', 'warehouse_cluster', 'customer_cluster'], as_index=False)\
                        .agg({'loss_from_return':'sum'}).sort_values('loss_from_return', ascending=False)
Return_loss

,date_order,number_order,carrier_name,delivery_type,warehouse_cluster,customer_cluster,loss_from_return
2692,2021-02-23,3544314143,3PL 4,Самовывоз,Кластер Москва,Кластер Санкт-Петербург,"50,998.54"
2949,2021-02-26,3554906914,3PL 1,Курьер,Кластер Москва,Кластер Казань,"43,807.33"
2292,2021-02-19,346773377,3PL 3,Курьер,Кластер Московская область,Кластер Московская область,"36,345.37"
2979,2021-02-27,352797427,3PL 4,Самовывоз,Кластер Санкт-Петербург,Кластер Центр,"29,726.63"
1291,2021-02-04,3312071441,3PL 4,Курьер,Кластер Казань,Кластер Москва,"25,700.60"
...,...,...,...,...,...,...,...
1366,2021-02-05,3367066914,3PL 2,Курьер,Кластер Казань,Кластер Казань,0.00
1367,2021-02-05,3374697145,Самодоставка,Курьер,Кластер Московская область,Кластер Москва,0.00
1368,2021-02-05,33141074714,3PL 1,Самовывоз,Кластер Москва,Кластер Санкт-Петербург,0.00
1369,2021-02-05,33141199914,Самодоставка,Курьер,Кластер Северо-Запад,Кластер Центр,0.00


Объединим возвраты и выручку

In [192]:
GMV_D_R=GMV_Accepted_merged.merge(Return_loss, on=['date_order', 'number_order', 'carrier_name', 'delivery_type', \
                                                   'warehouse_cluster','customer_cluster'], how='left')\
                    .sort_values('loss_from_return', ascending=False)
GMV_D_R['GMV_D_R']=GMV_D_R['GMV_Accepted']-GMV_D_R['loss_from_return']
pd.options.display.float_format ='{:,.2f}'.format

In [193]:
GMV_D_R=GMV_D_R[['date_order', 'number_order', 'carrier_name', 'delivery_type', \
                 'warehouse_cluster', 'customer_cluster', 'GMV_Accepted', 'loss_from_return', 'GMV_D_R']] 
GMV_D_R.head()

,date_order,number_order,carrier_name,delivery_type,warehouse_cluster,customer_cluster,GMV_Accepted,loss_from_return,GMV_D_R
2666,2021-02-23,3544314143,3PL 4,Самовывоз,Кластер Москва,Кластер Санкт-Петербург,"50,998.54","50,998.54",0.00
2907,2021-02-26,3554906914,3PL 1,Курьер,Кластер Москва,Кластер Казань,"43,807.33","43,807.33",0.00
2284,2021-02-19,346773377,3PL 3,Курьер,Кластер Московская область,Кластер Московская область,"36,345.37","36,345.37",0.00
3020,2021-02-27,352797427,3PL 4,Самовывоз,Кластер Санкт-Петербург,Кластер Центр,"29,726.63","29,726.63",-0.00
1308,2021-02-04,3312071441,3PL 4,Курьер,Кластер Казань,Кластер Москва,"25,700.60","25,700.60",0.00


In [194]:
GMV_D_R.to_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-s-androsova-23/Final_project_рабочая/GMV_D_R.csv')
# выгужу файл для построения графиков в Табло

Визуализация метрик в Табло

https://public.tableau.com/authoring/Project_Androsova_2022/Ozonmetrcicsdashboard_1#1